In [2]:
!curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=1AvU1UDSeg4uWfPA8JynjrrNnvg8oa0AZ" > /tmp/intermezzo.html
!curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > dataset.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3259    0  3259    0     0  43453      0 --:--:-- --:--:-- --:--:-- 43453
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   408    0   408    0     0   5230      0 --:--:-- --:--:-- --:--:--  5230
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  236M    0  236M    0     0   127M      0 --:--:--  0:00:01 --:--:--  158M


In [3]:
!pip install rank_bm25

In [4]:
## Downloading the sent2vec model
!curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=1PGrbSf7mksHbYAmTpOBEl6nQl7din5Vj" > /tmp/intermezzo.html
!curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > sent2vec_model.bin
## Install sent2vec
## Installing sent2vec
!git clone https://github.com/epfml/sent2vec.git
!cd sent2vec &&  make && pip install . 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3094    0  3094    0     0  30940      0 --:--:-- --:--:-- --:--:-- 30940
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 64788    0 64788    0     0   569k      0 --:--:-- --:--:-- --:--:--  569k
fatal: destination path 'sent2vec' already exists and is not an empty directory.
make: Nothing to be done for 'opt'.
Processing /content/sent2vec
  Created wheel for sent2vec: filename=sent2vec-0.0.0-cp36-cp36m-linux_x86_64.whl size=1126012 sha256=088accbbb8191896b10219cc2a573f46257b6aad207c5e9912f665e64d2a7980
  Stored in directory: /tmp/pip-ephem-wheel-cache-7nbe2ef9/wheels/b1/65/d7/f2c679e0086741bea69f0c4478303b6506bbf4c78dea82f565
Successf

In [5]:
from parsing import parse_exemple_file, get_dataset
df = parse_exemple_file()
df.head()

/content/parsing.py:12: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  header=0, index_col=False, dtype="str",  keep_default_na=False)


,doc_id,doi,year,title,journal_name,authors,abstract,fullbody
0,200,10.5812/jjm.33498,2016,Study of Genetic Variation of Leishmania majo...,Jundishapur Journal of Microbiology,Mansour Dabirzadeh\tMohammad Hashemi\tYahya Ma...,Background Zoonotic cutaneous leishmaniasis (Z...,1. Background Zoonotic cutaneous leishmaniasis...
1,201,10.5812/jjm.32388,2016,Characterization of Oxacillinase and Metallo-β...,Jundishapur Journal of Microbiology,Saeed Shoja\tMojtaba Moosavian\tSoodabeh Rosta...,Background: Carbapenem resistant Acinetobacter...,"1. Background In recent years, Acinetobacter b..."
2,202,10.5812/jjm.28291,2016,Staphylococcus aureus Isolates Carrying Panto...,Jundishapur Journal of Microbiology,Laleh Shariati\tMajid Validi\tAli Mohammad Has...,Background: A diversity of virulence factors w...,1. Background Staphylococcus aureus is a commo...
3,203,10.5812/jjm.10168,2014,In Vitro Study of Potentially Probiotic lacti...,Jundishapur Journal of Microbiology,Hooshang Niazi Amraii\tHamid Abtahi\tParvaneh ...,Background: Probiotic microorganisms are selec...,1. Background Great attention is currently dra...
4,204,10.5812/jjm.17824,2014,Phenotypic Characteristics and Probiotic Poten...,Jundishapur Journal of Microbiology,Nasrin Noohi\tNasrin Noohi\tGholamhosein Ebrah...,"Background: Lactic acid bacteria, especially L...",1. Background The normal flora bacteria in int...


In [6]:
ds = get_dataset(df)
print( "total number of title-abstract pairs:", len(ds) )
print( "First example in the dataset:\nTitle:\t%s\nAbstract:\t%s"%( ds[0][0], ds[0][1] ) )

total number of title-abstract pairs: 10000
First example in the dataset:
Title:	Study of Genetic Variation of  Leishmania major  Based on Internal Transcribed Spacer 1 (ITS1) in Chabahar, Iran
Abstract:	Background Zoonotic cutaneous leishmaniasis (ZCL) is polymorphic disease that may show various clinical manifestations. Objectives This study investigates the determination of genetic variation within the species of Leishmania major isolates from new cases in Chabahar, a port city in Southeast Iran (situated at the Iran-Pakistan border). Migration in this region indicates that leishmaniasis is spreading gradually, and a new micro-habitat focus appears each year. Materials and Methods A variety of nucleic acid detection methods that target both DNA and RNA have been developed. The restriction fragment length polymorphism analysis of amplified internal transcribed spacer 1 with polymerase chain reaction (ITS1-RFLP PCR) assay is a multipurpose tool for the diagnosis of Leishmania from cli

In [0]:
import nltk
from nltk import RegexpTokenizer
# This tokenizer is simply to use a regular expression "\w+" to remove all characters which are not from the set [0-9A-Za-z]
# This is not the best one but is very fast. Explore the nltk package for more powerful tokenizer!
tokenizer = RegexpTokenizer( "\w+" )

def tokenizer_fn(doc):
    return doc.split()

In [0]:
for record in ds:
    record[0] = " ".join(tokenizer.tokenize( record[0].lower()  ))
    record[1] = " ".join(tokenizer.tokenize( record[1].lower()  ))

In [0]:
queries, corpus = zip( *ds )

In [0]:
from ranking import Bm25Ranker
bm25 = Bm25Ranker(corpus, tokenizer_fn)

In [11]:
query = queries[0]
doc_scores = bm25.score_query(query)
doc_scores

array([75.27870625, 17.13461739, 15.65675489, ...,  0.        ,
        0.        ,  0.        ])

In [12]:
Knn_ids = bm25.knn_prediction(query)
Knn_ids

array([   0,  363, 7804,   19,  331, 7825,  439,  386,  141, 2090, 3164,
        201, 2399, 4450,  737,   41, 5338,   23,  303,  432])

In [0]:
from scoring import ml_score, plot_ml_curve, plot_ml_histograms

In [0]:
prediction = bm25.batch_knn_prediction(queries)

In [0]:
real_query_ids = [i for i in range(len(queries))]
ml_score(real_query_ids, prediction, 1)

In [0]:
ml_score(real_query_ids, prediction, 20)

In [0]:
plot_ml_curve(real_query_ids, prediction)

In [0]:
plot_ml_histograms(real_query_ids, prediction)

In [0]:
from ranking import Sent2VecRanker
model = Sent2VecRanker(corpus)

In [0]:
title_list , abstract_list = zip(*ds)
query = title_list[0]
print("Query ID:\t%d"%( 0 ))
print("20 closest documents ids:", model.knn_prediction(query))

In [0]:
prediction = model.batch_knn_prediction(queries)

In [0]:
real_query_ids = [i for i in range(len(queries))]
print(ml_score(real_query_ids, prediction, 1))
print(ml_score(real_query_ids, prediction, 20))

In [0]:
plot_ml_curve(real_query_ids, prediction)

In [0]:
plot_ml_histograms(real_query_ids, prediction)

In [0]:
from filter import *
from keyword_selection import *

In [0]:
#These lines are equivalent but the builder makes things much easier
#corpus_filter = CorpusFilter(corpus, tokenizer_fn, SelectKKeywordSelector(TfidfKeywordScorer(InvertedIndex(corpus, tokenizer_fn), len(corpus)), 3), OrFilterStrategy())
corpus_filter = CorpusFilterBuilder(corpus, tokenizer_fn).set_k_keyword_selector(3).build()

In [0]:
from ranking import Sent2VecHybridRanker
hybrid_ranker = Sent2VecHybridRanker(corpus, corpus_filter)

In [0]:
title_list , abstract_list = zip(*ds)
query = title_list[0]
print("Query ID:\t%d"%( 0 ))
print("20 closest documents ids:", hybrid_ranker.knn_prediction(query))

In [0]:
prediction = hybrid_ranker.batch_knn_prediction(queries)

In [0]:
real_query_ids = [i for i in range(len(queries))]
print(ml_score(real_query_ids, prediction, 1))
print(ml_score(real_query_ids, prediction, 20))

In [0]:
plot_ml_curve(real_query_ids, prediction)

In [0]:
plot_ml_histograms(real_query_ids, prediction)